In [137]:
from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB(fit_prior=False)

from sklearn.ensemble import AdaBoostClassifier
adb_classfier = AdaBoostClassifier(n_estimators=50)

from sklearn.svm import SVC
svc_classifier = SVC()

from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(criterion="entropy", max_depth=5)

from sklearn.neural_network import MLPClassifier
nn_clf = MLPClassifier(hidden_layer_sizes=(5,5,5))

In [138]:
import pandas as pd
import numpy as np

In [139]:
input_data = pd.read_csv('/Users/anwar/java_workspace/Graph-Mining-Project/output/graph_features_max_depth_4/feature_vector_complete.csv')
input_data = input_data.fillna(np.mean(input_data['D2VAvgCosDist']))
positive_cases = input_data[input_data['Cited'] == 1]
unlabled_cases = input_data[input_data['Cited'] == 0]

In [140]:
P = positive_cases.reset_index(drop=True)
P_hold_out = P.sample(frac=0.15)
P = P.drop(P_hold_out.index)
P.head()
P_hold_out.head()

U = unlabled_cases.reset_index(drop=True)
U.Cited.replace(0,-1, inplace = True)
U_hold_out = U.sample(frac=0.025)
U = U.drop(U_hold_out.index)
U.head()
U_hold_out.head()

X_input = P.append(U)
X_input = X_input.reset_index(drop= True)
y_input = X_input['Cited']
X_input = X_input.drop(['Cited', 'AuthorId', 'ArticleId'], axis=1)
X_input.ShortestDistance.replace(-1, 9999, inplace=True)
X_input.head()

,ShortestDistance,RandomWalkProbability,CurrentScoringMethod,PathLength=1,PathLength=2,PathLength=3,PathLength=4,N2VAvgCosDist,D2VAvgCosDist
0,2,0.032341,0.009708,0,5,81,2676,0.386456,0.295523
1,2,0.044821,0.015211,0,11,84,2001,0.355503,0.268981
2,2,0.030942,0.010616,0,8,60,1482,0.290643,0.226678
3,2,0.003465,0.001258,0,1,2,97,0.278734,0.163345
4,2,0.003534,0.001278,0,1,2,61,0.232596,0.194628


In [141]:
classifier = nn_clf

In [142]:
isConverged = False
U_Input = U.reset_index(drop=True)
U_Input.ShortestDistance.replace(-1, 9999)
RN = U_Input
RN_threshold = 0.85
iterationCount = 0
while(not isConverged):
    X_input = P.append(RN)
    X_input = X_input.reset_index(drop= True)
    y_input = X_input['Cited']
    X_input = X_input.drop(['Cited', 'AuthorId', 'ArticleId'], axis=1)
    X_input.ShortestDistance.replace(-1, 9999, inplace=True)
    #print(y_input)
    classifier.fit(X_input, y_input)
    U_Input = U_Input.reset_index(drop=True)
    U_probabilities = classifier.predict_proba(U_Input.drop(['Cited', 'AuthorId', 'ArticleId'], axis=1))
    #print(U_probabilities)
    RN_index = np.where(U_probabilities[:,0]>RN_threshold)[0]
    #print(RN_index)
    RN = U_Input.iloc[RN_index,:]
    U_Input = U_Input.drop(RN_index)
    print("Number of RN- ", len(RN))
    iterationCount+=1
    RN_threshold *= 0.9
    if(len(RN) <1):
        isConverged = True
        print("NO RN Found")
    if(iterationCount >30):
        isConverged = True
        print("Max Iteration Count Reached")
        

Number of RN-  125541
Number of RN-  55
Number of RN-  0
NO RN Found


In [143]:
X_test = P_hold_out.append(U_hold_out)
X_test = X_test.reset_index(drop=True)
Y_test = X_test['Cited']
X_test = X_test.drop(['Cited', 'AuthorId', 'ArticleId'],  axis =1)

Y_test_predict = classifier.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(Y_test, Y_test_predict))

             precision    recall  f1-score   support

       -1.0       0.00      0.00      0.00      4000
        1.0       0.13      1.00      0.23       584

avg / total       0.02      0.13      0.03      4584



/Users/anwar/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [144]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_test_predict,labels=[-1,1])

array([[   0, 4000],
       [   0,  584]])

In [145]:
classifier.classes_

array([-1.,  1.])

### 